In [4]:
import MySQLdb as mdb
import sys
sys.path.append("../src/")
from src.fingerprint import Fingerprint
from src import CSV_DELIMITER
import pandas as pd
DATA_PATH_REL = "../../data/"

In [5]:
ATTRIBUTES_TO_EXCLUDE = [Fingerprint.COUNTER, Fingerprint.CREATION_TIME, Fingerprint.END_TIME, Fingerprint.CONSISTENT]

In [6]:
df = pd.read_csv(DATA_PATH_REL + "processed/fingerprint_dataset.csv", delimiter=CSV_DELIMITER, engine="python")

In [ ]:
fingerprints = get_fingerprints_experiments(cur, 10, Fingerprint.MYSQL_ATTRIBUTES, limitTest=60000)

In [13]:
number_distinct_values = analyse.get_number_distinct_values(cur, Fingerprint.MYSQL_ATTRIBUTES)
for attribute, number in number_distinct_values.items():
    print(attribute, number)

addressHttp 23303
languageHttp 655
IEDataJS 1
canvasJSHashed 4002
platformFlash 291
hostHttp 3
platformJS 33
creationDate 11886
cookiesJS 2
userAgentHttp 3551
resolutionFlash 311
dntJS 2
timezoneJS 35
endDate 11806
pluginsJS 5830
fontsFlash 5341
orderHttp 196
sessionJS 2
connectionHttp 2
counter 94767
resolutionJS 610
adBlock 2
id 5809
encodingHttp 18
localJS 2
acceptHttp 17
languageFlash 28


In [46]:
attributes_to_analyze = [attribute for attribute in df.columns.tolist() if attribute not in ATTRIBUTES_TO_EXCLUDE]
df_consistent = df[df[Fingerprint.CONSISTENT] == True ]

In [57]:
df_consistent.columns

Index(['counter', 'id', 'creationDate', 'endDate', 'addressHttp',
       'userAgentHttp', 'acceptHttp', 'connectionHttp', 'encodingHttp',
       'languageHttp', 'orderHttp', 'pluginsJSHashed', 'platformJS',
       'cookiesJS', 'dntJS', 'timezoneJS', 'resolutionJS', 'localJS',
       'sessionJS', 'canvasJSHashed', 'fontsFlashHashed', 'resolutionFlash',
       'languageFlash', 'platformFlash', 'browserFamily',
       'globalBrowserVersion', 'minorBrowserVersion', 'majorBrowserVersion',
       'os', 'rendererWebGLJS', 'vendorWebGLJS', 'consistent'],
      dtype='object')

In [45]:
df_consistent.head(5)

,counter,id,creationDate,endDate,addressHttp,userAgentHttp,acceptHttp,connectionHttp,encodingHttp,languageHttp,...,languageFlash,platformFlash,browserFamily,globalBrowserVersion,minorBrowserVersion,majorBrowserVersion,os,rendererWebGLJS,vendorWebGLJS,consistent
15,17,0f4f77e3-80da-4b4a-b9c0-f5e8c4239380,2015-07-27 11:00:00,None,24f5398bba446ce8dadfd3d23ca2f4e58895e76e,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,"text/html,application/xhtml+xml,application/xm...",close,"gzip, deflate, sdch","en-US,en;q=0.8,fr;q=0.6",...,en,Linux,Chrome,440,0,44,Linux,Mesa DRI Intel(R) Haswell Mobile,Intel Open Source Technology Center,True
17,19,74bf8f6f-0114-44b4-9bca-1cfc52ffc2d3,2015-07-27 12:00:00,None,24f5398bba446ce8dadfd3d23ca2f4e58895e76e,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,"text/html,application/xhtml+xml,application/xm...",close,"gzip, deflate, sdch","en-US,en;q=0.8,fr;q=0.6",...,en,Linux,Chrome,440,0,44,Linux,Mesa DRI Intel(R) Haswell Mobile,Intel Open Source Technology Center,True
18,20,df46c5fb-9889-4ff4-9f51-e5af591f3460,2015-07-27 12:00:00,None,24f5398bba446ce8dadfd3d23ca2f4e58895e76e,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,"text/html,application/xhtml+xml,application/xm...",close,"gzip, deflate, sdch","en-US,en;q=0.8,fr;q=0.6",...,en,Linux,Chrome,440,0,44,Linux,Mesa DRI Intel(R) Haswell Mobile,Intel Open Source Technology Center,True
20,22,d51d09c6-2596-4958-bfe7-0352af2aaccf,2015-07-27 12:00:00,2015-07-29 06:00:00,2515ee56512877237d467911f5c55be53eea3550,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,"text/html,application/xhtml+xml,application/xm...",close,"gzip, deflate, sdch","fr-FR,fr;q=0.8,en-US;q=0.6,en;q=0.4",...,en,Linux,Chrome,430,0,43,Linux,Not supported,Not supported,True
35,39,7c7f536a-055b-4b4c-b30f-2ae078797bea,2015-07-29 14:00:00,None,24f5398bba446ce8dadfd3d23ca2f4e58895e76e,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,"text/html,application/xhtml+xml,application/xm...",close,"gzip, deflate, sdch","en-US,en;q=0.8,fr;q=0.6",...,en,Linux,Chrome,440,0,44,Linux,Mesa DRI Intel(R) Haswell Mobile,Intel Open Source Technology Center,True


In [151]:
nb = df_consistent[Fingerprint.OS].value_counts()
nb

Windows 7        730
Windows 10       486
Linux            245
Mac OS X         170
Windows 8.1      163
Ubuntu           158
Windows XP        71
Chrome OS         40
Windows 8         27
Windows Vista     16
Other              9
OpenBSD            6
FreeBSD            3
NetBSD             1
Name: os, dtype: int64

In [100]:
df_consistent[df_consistent[Fingerprint.MAJOR_BROWSER_VERSION] == "None"]["counter"]

10095     12782
45033     57934
52521     67757
59961     77378
71595     93843
72843     95707
87820    118549
Name: counter, dtype: int64

In [101]:
df_consistent[df_consistent[Fingerprint.MAJOR_BROWSER_VERSION]== "None"]["userAgentHttp"]

10095    this is a test ua &lt;img src=x onerror= alert...
45033                       xChaos_Arachne/5.1.89;GPL,386+
52521    Mozilla/5.0 (PlayStation 4 3.15) AppleWebKit/5...
59961    Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:4...
71595                                                  NaN
72843                                                  NaN
87820                                                  NaN
Name: userAgentHttp, dtype: object

In [98]:
df_consistent[df_consistent[Fingerprint.COUNTER] == 47714]

,counter,id,creationDate,endDate,addressHttp,userAgentHttp,acceptHttp,connectionHttp,encodingHttp,languageHttp,...,languageFlash,platformFlash,browserFamily,globalBrowserVersion,minorBrowserVersion,majorBrowserVersion,os,rendererWebGLJS,vendorWebGLJS,consistent
37496,47714,3371db88-3ffe-4c67-8187-ec262ab62112,2016-06-24 18:00:00,None,5c1cb9cba5e0e48ebd1237cc58f936ded716b681,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2...,"text/html,application/xhtml+xml,application/xm...",close,"gzip,deflate,sdch","en-US,en;q=0.8",...,Flash not detected,Flash not detected,Chrome,432357,2357,43,Mac OS X,Not supported,Not supported,True


In [123]:
df_consistent.nunique()


counter                 2125
id                      2125
creationDate            1850
endDate                  128
addressHttp             1890
userAgentHttp            626
acceptHttp                 6
connectionHttp             1
encodingHttp              10
languageHttp             219
orderHttp                 44
pluginsJSHashed          869
platformJS                16
cookiesJS                  2
dntJS                      2
timezoneJS                29
resolutionJS             135
localJS                    2
sessionJS                  2
canvasJSHashed           480
fontsFlashHashed         611
resolutionFlash           74
languageFlash             22
platformFlash             76
browserFamily             17
globalBrowserVersion      63
minorBrowserVersion       18
majorBrowserVersion       38
os                        14
rendererWebGLJS          223
vendorWebGLJS             13
consistent                 1
dtype: int64

In [148]:
tmp = df_consistent['globalBrowserVersion'].value_counts()
tmp[tmp.values == 1]
len(tmp[tmp.values == 1])

24

In [159]:
nb[:4]

Windows 7     730
Windows 10    486
Linux         245
Mac OS X      170
Name: os, dtype: int64

In [174]:
nb_combine = df_consistent.groupby(['browserFamily','os']).size()
nb_combine

browserFamily                os           
Chrome                       Chrome OS         40
                             Linux            107
                             Mac OS X          87
                             OpenBSD            3
                             Windows 10       201
                             Windows 7        248
                             Windows 8          7
                             Windows 8.1       31
                             Windows Vista      3
                             Windows XP         4
Chromium                     Ubuntu            33
Dragon                       Windows 10         2
                             Windows 7          1
Epiphany                     Ubuntu             2
Firefox                      FreeBSD            3
                             Linux             98
                             Mac OS X          80
                             NetBSD             1
                             OpenBSD            3
       